# Deep Convolutional Generative Adversarial Networks (DCGAN)

Neste notebook, exploraremos a arquitetura DCGAN, uma das primeiras e mais influentes arquiteturas de redes adversariais generativas que utiliza majoritariamente camadas convolucionais. As DCGANs trouxeram um conjunto de diretrizes arquitetônicas que não apenas estabilizaram o treinamento de GANs, mas também permitiram a geração de imagens com maior qualidade e resolução. Assumimos que os conceitos fundamentais de GANs, como o jogo minimax entre um Gerador e um Discriminador, já foram compreendidos. Aqui, focaremos nos detalhes técnicos e na implementação que tornam a DCGAN eficaz.

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Preparação dos Dados (MNIST)

Para treinar nossa DCGAN, utilizaremos o dataset MNIST. As imagens serão redimensionadas para 32x32 para facilitar uma arquitetura com múltiplas camadas de convolução transposta que dobram a dimensão espacial. Além disso, normalizaremos os pixels das imagens para o intervalo `[-1, 1]`, que corresponde à faixa da função de ativação `Tanh` na camada de saída do nosso Gerador.

In [ ]:
batch_size = 128
image_size = 32

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # Normaliza para [-1, 1]
])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Número de batches de treino: {len(train_dataloader)}")
print(f"Número de batches de teste: {len(test_dataloader)}")

### O Gerador

O Gerador ($G$) tem a função de mapear um vetor de ruído do espaço latente ($z \in \mathbb{R}^{100}$) para o espaço de dados ($G(z) \in \mathbb{R}^{32 \times 32}$). Ele realiza essa tarefa através de uma série de camadas de convolução transposta (`ConvTranspose2d`), que progressivamente aumentam a dimensão espacial dos mapas de características, enquanto reduzem sua profundidade (número de canais). A Batch Normalization é aplicada após cada convolução transposta para estabilizar o fluxo de gradientes.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels, features_g):
        super().__init__()
        self.net = nn.Sequential(
            # Entrada: Vetor de ruído (latent_dim)
            # Saída: features_g*8 x 4 x 4
            nn.ConvTranspose2d(latent_dim, features_g * 8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            
            # Saída: features_g*4 x 8 x 8
            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            
            # Saída: features_g*2 x 16 x 16
            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            
            # Saída: img_channels x 32 x 32
            nn.ConvTranspose2d(features_g * 2, img_channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh() # Normaliza a saída para [-1, 1]
        )

    def forward(self, input):
        return self.net(input)

### O Discriminador

O Discriminador ($D$) é uma rede neural convolucional padrão que atua como um classificador binário. Sua tarefa é receber uma imagem (seja ela real do dataset ou falsa, gerada por $G$) e produzir um escalar que representa a probabilidade da imagem ser real. A arquitetura espelha a do Gerador, utilizando convoluções com `stride=2` para reduzir a dimensão espacial, `LeakyReLU` como função de ativação para permitir o fluxo de gradientes mesmo para entradas negativas, e `Batch Normalization`. A camada final utiliza a função `Sigmoid` para mapear a saída para o intervalo $[0, 1]$.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, img_channels, features_d):
        super().__init__()
        self.net = nn.Sequential(
            # Entrada: img_channels x 32 x 32
            # Saída: features_d x 16 x 16
            nn.Conv2d(img_channels, features_d, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Saída: features_d*2 x 8 x 8
            nn.Conv2d(features_d, features_d * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Saída: features_d*4 x 4 x 4
            nn.Conv2d(features_d * 2, features_d * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Saída: 1 x 1 x 1 (probabilidade)
            nn.Conv2d(features_d * 4, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.net(input)

### Inicialização de Pesos e Instanciação

O artigo da DCGAN sugere que os pesos das camadas `Conv` e `ConvTranspose` devem ser inicializados a partir de uma distribuição Normal com média 0 e desvio padrão 0.02. Esta prática ajuda a prevenir que os gradientes saturem ou desapareçam no início do treinamento.

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Instanciação dos modelos
features_g = 16
features_d = 16
latent_dim = 100
img_channels = 1

generator = Generator(latent_dim, img_channels, features_g).to(device)
generator.apply(weights_init)

discriminator = Discriminator(img_channels, features_d).to(device)
discriminator.apply(weights_init)

print("Generator Architecture:\n", generator)
print("\nDiscriminator Architecture:\n", discriminator)

### Função de Perda e Otimizadores

O treinamento de GANs envolve a otimização de uma função de perda minimax, que para o caso discreto é a Entropia Cruzada Binária (`Binary Cross-Entropy`).

A função objetivo do Discriminador é maximizar a probabilidade de classificar corretamente as imagens reais e falsas. A do Gerador é minimizar a probabilidade do Discriminador classificar suas saídas como falsas. A função de perda é:

$$
\mathcal{L}(D, G) = \mathbb{E}_{x \sim p_{data}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_{z}(z)}[\log(1 - D(G(z)))]
$$

Utilizaremos o otimizador Adam separadamente para o Gerador e para o Discriminador, conforme recomendado no artigo, com uma taxa de aprendizado de 0.0002 e parâmetros de beta1=0.5 para maior estabilidade.

In [ ]:
# Otimizadores e Função de Perda
lr = 0.0002
beta1 = 0.5

optimizer_g = Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_d = Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
criterion = nn.BCELoss()

# Vetor de ruído fixo para visualização da evolução do gerador
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

### O Loop de Treinamento

O treinamento é um processo iterativo onde, a cada passo, realizamos duas atualizações: uma para o Discriminador e outra para o Gerador.

1.  **Treinamento do Discriminador**:
    * Alimentamos o Discriminador com um batch de imagens reais do dataset. Calculamos a perda ($D(x)$) em relação a labels "reais" (valor 1).
    * Geramos um batch de imagens falsas usando o Gerador ($G(z)$). Alimentamos o Discriminador com essas imagens. Calculamos a perda ($D(G(z))$) em relação a labels "falsas" (valor 0).
    * Somamos as duas perdas e atualizamos os pesos do Discriminador via backpropagation.

2.  **Treinamento do Gerador**:
    * Geramos um novo batch de imagens falsas ($G(z)$).
    * Passamos essas imagens pelo Discriminador.
    * Calculamos a perda do Gerador baseando-se na saída do Discriminador, mas desta vez, em relação a labels "reais" (valor 1). O objetivo do Gerador é "enganar" o Discriminador, fazendo-o classificar as imagens falsas como reais.
    * Atualizamos os pesos do Gerador.

In [ ]:
num_epochs = 10
plot_interval = 1
g_losses, d_losses = [], []

for epoch in range(num_epochs):
    generator.train()
    discriminator.train()
    g_loss, d_loss = 0, 0

    for real_imgs, _ in train_dataloader:
        real_imgs = real_imgs.to(device)
        N = real_imgs.size(0)
        valid = torch.ones(N, device=device)
        fake = torch.zeros(N, device=device)

        # --- Discriminador ---
        optimizer_d.zero_grad()
        loss_d_real = criterion(discriminator(real_imgs).view(-1), valid)
        noise = torch.randn(N, latent_dim, 1, 1, device=device)
        fake_imgs = generator(noise).detach()
        loss_d_fake = criterion(discriminator(fake_imgs).view(-1), fake)
        loss_d = 0.5 * (loss_d_real + loss_d_fake)
        loss_d.backward()
        optimizer_d.step()

        # --- Gerador ---
        optimizer_g.zero_grad()
        noise = torch.randn(N, latent_dim, 1, 1, device=device)
        gen_imgs = generator(noise)
        loss_g = criterion(discriminator(gen_imgs).view(-1), valid)
        loss_g.backward()
        optimizer_g.step()

        d_loss += loss_d.item()
        g_loss += loss_g.item()

    g_losses.append(g_loss / len(train_dataloader))
    d_losses.append(d_loss / len(train_dataloader))

    if epoch % plot_interval == 0 or epoch == num_epochs - 1:
        print(f"[{epoch}/{num_epochs-1}] Loss D: {d_losses[-1]:.4f} | Loss G: {g_losses[-1]:.4f}")
        generator.eval()
        with torch.no_grad():
            noise = torch.randn(8, latent_dim, 1, 1, device=device)
            imgs = generator(noise).cpu()
        fig, axs = plt.subplots(2, 4, figsize=(8, 4))
        for i, ax in enumerate(axs.flat):
            ax.imshow(imgs[i].squeeze(), cmap="gray")
            ax.axis("off")
        plt.suptitle(f'Época {epoch}')
        plt.show()

### Análise dos Resultados

Após o treinamento, podemos analisar os resultados de duas formas principais: visualizando a curva de perda do Gerador e do Discriminador e observando as imagens geradas ao longo do tempo. Idealmente, as perdas de $D$ e $G$ devem convergir para um estado de equilíbrio, embora na prática elas flutuem bastante. A análise mais importante é a qualidade visual das imagens geradas.

In [ ]:
# Plotar o gráfico de perdas
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(g_losses, label="G")
plt.plot(d_losses, label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Exercícios

### Exercício 1
Modifique a dimensão do espaço latente (`latent_dim`). Teste valores como 50 e 200. Adicionalmente, altere a taxa de aprendizado (`lr`) para valores maiores (e.g., 0.002) e menores (e.g., 0.00005). Observe e documente como essas mudanças afetam a velocidade de convergência do treinamento, a estabilidade das perdas do gerador e do discriminador, e a qualidade visual final das imagens geradas.

### Exercício 2
Adapte o código para treinar a DCGAN no dataset Fashion-MNIST, que também possui imagens em escala de cinza de 28x28. Como o Fashion-MNIST é visualmente mais complexo que o MNIST, analise se o modelo requer mais épocas para gerar imagens de boa qualidade.